In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [17]:
link = 'https://social.hse.ru/ma'
html = requests.get(link)
soup = BeautifulSoup(html.text)
print(soup)

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="ru">
<head>
<title>Магистратура – Факультет социальных наук – Национальный исследовательский университет «Высшая школа экономики»</title>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Магистратура" property="og:title"/>
<meta content="Магистратура" name="mrc__share_title"/>
<meta content="Магистратура" itemprop="name"/><meta content="https://www.hse.ru/images/fb/hse_ru_thumb.jpg" property="og:image"/>
<meta content="https://www.hse.ru/images/fb/hse_ru_thumb.jpg" itemprop="image"/>
<link href="https://www.hse.ru/images/fb/hse_ru_thumb.jpg" rel="image_src"/><meta content="https://social.hse.ru/ma" property="og:url"/><meta content="website" property="og:type"/>
<link href="/f/src/global/i/favicon/favicon_ios_180x180.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/f/src/global/i/favicon

In [18]:
tutlink = 'tutors'
dictma = {}
for h2 in soup.find_all('h2', class_='-3 pk-section__title builder-section__title'):
    key = h2.get_text(strip=True)
    items = []
    for el in h2.next_elements:
        if getattr(el, 'name', None) == 'h2':
            break
        if getattr(el, 'name', None) == 'h3' and 'pk-color-card__title' in (el.get('class') or []):
            items.append(el.get_text(strip=True))
    dictma[key] = items
dictma

{'Политология': ['Прикладная политология',
  'Политика. Экономика. Философия',
  'Современные социальные науки в преподавании обществознания в школе'],
 'Социология': ['Демография',
  'Социальный анализ рынков на цифровых и опросных данных',
  'Социология публичной сферы и цифровая аналитика',
  'Сравнительные социальные исследования / Comparative Social Research',
  'Комплексный социальный анализ'],
 'Государственное и муниципальное управление': ['Государственное и муниципальное управление',
  'Управление и экономика здравоохранения',
  'Население и развитие / Population and development'],
 'Прикладная математика и информатика': ['Аналитика данных и прикладная статистика / Data Analytics and Social Statistics'],
 'Психология': ['Персонология. Консультативная психология и психотерапия',
  'Позитивная психология',
  'Прикладная социальная психология',
  'Психология в бизнесе',
  'Системная семейная психотерапия',
  'Психоанализ и психоаналитическая психотерапия',
  'Психоанализ и психоа

***На этапе сбора данных нам нужен трехуровневый словарь: название "направлений" (первый уровень), их магистерских программ (второй уровень) и ссылок на страницы с их преподавателями (третий уровень)***

In [19]:
for h2_title, programs in dictma.items():
    h2_tag = soup.find(lambda t: t.name == 'h2' and t.get_text(strip=True) == h2_title.strip())
    tutor_url = ''
    if h2_tag:
        for el in h2_tag.next_elements:
            name = getattr(el, 'name', None)
            if name == 'h2':
                break
            if name == 'a' and 'pk-color-card__button' in (el.get('class') or []):
                href = el.get('href') or ''
                tutor_url = (href + tutlink) if href else ''
                break
    dictma[h2_title] = {prog: tutor_url for prog in programs}

In [20]:
dictma

{'Политология': {'Прикладная политология': 'https://www.hse.ru/ma/ps/tutors',
  'Политика. Экономика. Философия': 'https://www.hse.ru/ma/ps/tutors',
  'Современные социальные науки в преподавании обществознания в школе': 'https://www.hse.ru/ma/ps/tutors'},
 'Социология': {'Демография': 'https://www.hse.ru/ma/demography/tutors',
  'Социальный анализ рынков на цифровых и опросных данных': 'https://www.hse.ru/ma/demography/tutors',
  'Социология публичной сферы и цифровая аналитика': 'https://www.hse.ru/ma/demography/tutors',
  'Сравнительные социальные исследования / Comparative Social Research': 'https://www.hse.ru/ma/demography/tutors',
  'Комплексный социальный анализ': 'https://www.hse.ru/ma/demography/tutors'},
 'Государственное и муниципальное управление': {'Государственное и муниципальное управление': 'https://www.hse.ru/ma/gmu/tutors',
  'Управление и экономика здравоохранения': 'https://www.hse.ru/ma/gmu/tutors',
  'Население и развитие / Population and development': 'https://ww

In [15]:
#Для себя сделал пример, чтобы понимать, как выстраивать цикл
dictexample = {}
link = 'https://www.hse.ru/ma/ps/tutors'
html = requests.get(link)
soup = BeautifulSoup(html.text)
newdict = {}
for i in soup.find_all('a', class_='b link_dark2 link'):
    newdict[i.text] = 'https://www.hse.ru' + i.get('href')
dictexample['Прикладная политология'] = newdict

{'Прикладная политология': {'Ахременко Андрей Сергеевич': 'https://www.hse.ru/mirror/org/persons/redir/24114108?pl=',
  'Баландин Юлиан Андреевич': 'https://www.hse.ru/mirror/org/persons/redir/137284738?pl=',
  'Беленков Вадим Евгеньевич': 'https://www.hse.ru/mirror/org/persons/redir/225516498?pl=',
  'Гайворонский Юрий Олегович': 'https://www.hse.ru/mirror/org/persons/redir/25907562?pl=',
  'Горельский Илья Евгеньевич': 'https://www.hse.ru/mirror/org/persons/redir/137293632?pl=',
  'Гулевич Ольга Александровна': 'https://www.hse.ru/mirror/org/persons/redir/66960?pl=',
  'Карпич Юлия Владимировна': 'https://www.hse.ru/mirror/org/persons/redir/190892405?pl=',
  'Каспэ Святослав Игоревич': 'https://www.hse.ru/mirror/org/persons/redir/67280?pl=',
  'Конча Валерия': 'https://www.hse.ru/mirror/org/persons/redir/401611351?pl=',
  'Кручинская Екатерина Владиславовна': 'https://www.hse.ru/mirror/org/persons/redir/160976594?pl=',
  'Макаренко Борис Игоревич': 'https://www.hse.ru/mirror/org/pers

In [23]:
newdictma = {}
for napr, progs in dictma.items():
    newdictt = {}
    for prog, linkk in progs.items():
        link = linkk
        html = requests.get(link)
        soup = BeautifulSoup(html.text)
        newdict = {}
        for i in soup.find_all('a', class_='b link_dark2 link'):
            newdict[i.text] = 'https://www.hse.ru' + i.get('href')
        newdictt[prog] = newdict
    newdictma[napr] = newdictt
newdictma

{'Политология': {'Прикладная политология': {'Ахременко Андрей Сергеевич': 'https://www.hse.ru/mirror/org/persons/redir/24114108?pl=',
   'Баландин Юлиан Андреевич': 'https://www.hse.ru/mirror/org/persons/redir/137284738?pl=',
   'Беленков Вадим Евгеньевич': 'https://www.hse.ru/mirror/org/persons/redir/225516498?pl=',
   'Гайворонский Юрий Олегович': 'https://www.hse.ru/mirror/org/persons/redir/25907562?pl=',
   'Горельский Илья Евгеньевич': 'https://www.hse.ru/mirror/org/persons/redir/137293632?pl=',
   'Гулевич Ольга Александровна': 'https://www.hse.ru/mirror/org/persons/redir/66960?pl=',
   'Карпич Юлия Владимировна': 'https://www.hse.ru/mirror/org/persons/redir/190892405?pl=',
   'Каспэ Святослав Игоревич': 'https://www.hse.ru/mirror/org/persons/redir/67280?pl=',
   'Конча Валерия': 'https://www.hse.ru/mirror/org/persons/redir/401611351?pl=',
   'Кручинская Екатерина Владиславовна': 'https://www.hse.ru/mirror/org/persons/redir/160976594?pl=',
   'Макаренко Борис Игоревич': 'https://

In [28]:
import json
with open('newdictma3.js', 'w', encoding='utf-8') as f:
    f.write('window.DATA = ')
    json.dump(newdictma, f, ensure_ascii=False)
    f.write(';')